In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
np.random.seed(1)

In [3]:
nTrials = 9600
nUnits = 35
nConditions = 12
nPara = 6
trials_p_block = 50
nAverage = 300
nRandom = 100
nBlocks = 32
Amax = 0.5
iSubject = 1
bias_update = 0.02

w_min = -1
w_max = 1

In [4]:
activation = pd.read_csv('Cache.csv', header=None).values.reshape((nConditions, nRandom, nUnits))

df = pd.read_csv('Data_withfeedback.csv')[iSubject * 9600 : (iSubject + 1) * 9600]
condition = df['Condition'].values-1
correct = df['Correct'].values
contrast = df['Contrast'].values
congruent = df['Congruent'].values

sign = [-1,1,-1,1,-1,1,-1,1,-1,1,-1,1]
sign_seq = [sign[condi] for condi in condition]

li = np.intersect1d(np.where(contrast==0.10),np.where(congruent==0))
mi = np.intersect1d(np.where(contrast==0.15),np.where(congruent==0))
hi = np.intersect1d(np.where(contrast==0.23),np.where(congruent==0))
lc = np.intersect1d(np.where(contrast==0.10),np.where(congruent==1))
mc = np.intersect1d(np.where(contrast==0.15),np.where(congruent==1))
hc = np.intersect1d(np.where(contrast==0.23),np.where(congruent==1))

noise_d = np.random.normal(0, 1, size=(nTrials,nAverage))
noise_r = np.random.normal(0, 1, size=(nTrials,nAverage,nUnits))
random_seq = np.random.randint(0,100,(nTrials,nAverage))

In [5]:
acti_seq = np.zeros((nTrials, nAverage, nUnits))
for i in range(nTrials):
    acti_seq[i] = activation[condition[i]][random_seq[i]]

In [6]:
def simulation(para):
    # initial weight and bias
    weight = np.tile(np.array([-45, -30, -15, 0, 15, 30, 45]) / 30,(nAverage, 5)) * para[5]
    bias = np.tile(0, nAverage)
    
    # arr for recording iteration
    decision_arr = np.zeros((nTrials, nAverage))
    bias_arr = np.zeros((nTrials, nAverage))
    weight_arr = np.zeros((nTrials, nAverage, nUnits))

    # initial sequence
    noise_d_seq =  noise_d * para[3]
    noise_r_seq =  noise_r * para[4]
    acti_withnoise = acti_seq + noise_r_seq
    acti_normalized =((1-np.exp(-para[2] * np.where(acti_withnoise > 0,acti_withnoise,0))) / (1 + np.exp(-para[2] * np.where(acti_withnoise > 0,acti_withnoise,0))))*Amax
    
    # performing iteration
    for i in range(nTrials):
        # simulate and record decision
        decision = np.sign(np.sum(acti_normalized[i] * weight,axis=1) - para[1] * bias + noise_d_seq[i])
        decision_arr[i] = decision
        
        # simulate and record bias
        bias = bias_update * decision + (1-bias_update) * bias
        bias_arr[i] = bias
        
        # simulate and record weight
        delta = acti_normalized[i] * para[0] * sign_seq[i] * Amax
        weight  = weight + (w_max-weight) * np.where(delta >= 0,delta,0) + (weight-w_min) * np.where(delta < 0,delta,0)
        weight_arr[i] = weight
    # average decision to get probability to be right
    prob = (((decision_arr.sum(axis = 1) / nAverage) * sign_seq) + 1) / 2
    
    # assign prob to 6 learning curves
    block_prob = np.sum(prob[[li,mi,hi,lc,mc,hc]].reshape((6,32,50)), axis=2)/trials_p_block
    return block_prob, bias_arr, weight_arr

In [7]:
lr_range = [0.0010,0.0015,0.0020,0.0025]
bias_range = [1.5,2.0,2.5,3.0]
gamma_range = [0.4,0.8,1.2,1.6]
noise_d_range = [0.12,0.20,0.28,0.36]
noise_r_range = [0.05,0.10,0.15,0.20]
w_init_range = [0.15,0.20,0.25,0.30]

search_grid = np.array(np.meshgrid(lr_range, bias_range, gamma_range, noise_d_range, noise_r_range, w_init_range,indexing='xy')).T.reshape(-1, nPara)

In [14]:
# generate weight change

def draw_w(para, iSubject):
    _, _, weight_arr = simulation(para)
    w = weight_arr.sum(axis=1)/nAverage
    #chagne of weight
    fig,((ax1, ax_none_1, ax2), (ax3, ax4, ax5), (ax6, ax7, ax_none_2)) = plt.subplots(3, 3, figsize=(12, 10))
    fig.delaxes(ax_none_1)
    fig.delaxes(ax_none_2)

    #weight change
    axes=[ax1, ax2]
    cycle = [w[:, np.arange(14, 21)], w[:, np.arange(28, 35)]]
    freqency=[2.0,4.0]
    ax1.set_position([0.15,0.65,0.3,0.3])
    ax2.set_position([0.55,0.65,0.3,0.3])

    for i in range(2):
        for j in range(7):
            axes[i].plot(cycle[i][:, j], color='black',  linewidth=0.5)
            axes[i].set_title(f'{freqency[i]} cycles/degree')
            axes[i].set_xticks([300,2700,5100,7500,9300])
            axes[i].set_ylim(-0.45,0.45)


    #contest switch
    axes=[ax3, ax4, ax5, ax6, ax7]
    marker=['o','^','None']
    label=['2.0 cyc/deg','2.8 cyc/deg','4.0 cyc/deg']
    weight_arr=[[],[],[],[],[]]
    context=['L','R','L','R','L']
    trial=np.array([299,2699,5099,7499,9299])
    for i in range(5):
        for j in range(3):
        #frequency 2.0, 2.8, 4.0
            weight_arr=[[],[],[],[],[]]
            weight_arr[i]=w[trial[i]][(j+2)*7+np.arange(7)]
            axes[i].plot(np.arange(-3,4)*15, weight_arr[i], marker=marker[j], markerfacecolor='none',markeredgewidth=1,markersize=5, color='black',label=label[j])
            axes[i].set_xticks([-45,-30,-15,0,15,30,45])
    ax7.legend(loc='lower right', bbox_to_anchor=(2,0.1), fontsize=15)
    fig.text(0.25, 0.05, 'Orientation', ha='center', va='center', fontsize=15)
    fig.text(0.5, 0.05, 'Orientation', ha='center', va='center', fontsize=15)
    fig.text(0.78, 0.32, 'Orientation', ha='center', va='center', fontsize=15)
    fig.text(0.06, 0.35, 'Weight', ha='center', va='center', rotation='vertical',fontsize=15)
    fig.text(0.06, 0.78, 'Weight', ha='center', va='center', rotation='vertical',fontsize=15)
    plt.savefig(f'results/weight/Subject_{iSubject}.png',bbox_inches='tight')
    plt.show()



In [ ]:
for iSubject in range(13):
    para = np.exp(pd.read_csv('results/HBM Solution_NUTS.csv')['mean'][78 + iSubject * 6 : 78 + (iSubject + 1) * 6].values)
    draw_w(para, iSubject = iSubject)
    

In [ ]:
para = np.exp(pd.read_csv('results/Population Solution_NUTS.csv')['mean'][78 + iSubject * 6 : 78 + (iSubject + 1) * 6].values)
draw_w(para, iSubject = 0)